In [2]:
import os

In [3]:
%pwd

'c:\\Users\\nikhil\\OneDrive\\Desktop\\ML Projects\\ipp\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\nikhil\\OneDrive\\Desktop\\ML Projects\\ipp'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    train_data_file: Path
    test_data_file: Path

In [18]:

from insurancePP.constants import *
from insurancePP.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        # logger.info(f"Root directory {self.config.artifacts_root} created successfully") 
        
   
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            train_data_file = config.train_data_file,
            test_data_file = config.test_data_file
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
import pandas as pd
from insurancePP.logging import logger
from insurancePP.utils.common import get_size
from sklearn.model_selection import train_test_split

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f'Downloaded {filename} to {self.config.local_data_file}')
        else:
            logger.info(f'File {self.config.local_data_file} already exists')


    def initiate_data_ingestion(self):
        logger.info("Entered inside the data ingestion component")

        try:
            data = pd.read_csv(self.config.local_data_file)
            logger.info("train test split initiated")

            train, test = train_test_split(data, test_size=0.2, random_state=50)
            train.to_csv(self.config.train_data_file)
            test.to_csv(self.config.test_data_file)
        except Exception as e:
            raise e


In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.initiate_data_ingestion()

except Exception as e:
    raise e

[2024-02-25 12:08:14,157 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-02-25 12:08:14,161 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-02-25 12:08:14,164 : INFO : common : directory artifacts created]
[2024-02-25 12:08:14,166 : INFO : common : directory artifacts/data_ingestion created]


[2024-02-25 12:08:15,563 : INFO : 489743891 : Downloaded artifacts/data_ingestion/data.csv to artifacts/data_ingestion/data.csv]
[2024-02-25 12:08:15,563 : INFO : 489743891 : Entered inside the data ingestion component]
[2024-02-25 12:08:15,592 : INFO : 489743891 : train test split initiated]
